In [1]:
#Import libraries
import cv2
import numpy as np
#Reading the video


vidcap = cv2.VideoCapture(0)
success,image = vidcap.read()
count = 0
success = True
idx = 0


In [2]:
pathVideo = r"Messi.mp4"
path = r"E:\ComputerVision\computer-vision\3_1YOLO\home\VideoTobe"
pathFoot = r"E:\ComputerVision\computer-vision\3_1YOLO\home\VideoTobe\ADemoV7.mp4"
import glob
v = glob.glob(path+'\*.mp4')
print(v)

['E:\\ComputerVision\\computer-vision\\3_1YOLO\\home\\VideoTobe\\ADemo3.mp4', 'E:\\ComputerVision\\computer-vision\\3_1YOLO\\home\\VideoTobe\\ADemoV7 (1).mp4', 'E:\\ComputerVision\\computer-vision\\3_1YOLO\\home\\VideoTobe\\ADemoV7 (2).mp4', 'E:\\ComputerVision\\computer-vision\\3_1YOLO\\home\\VideoTobe\\ADemoV7 (3).mp4', 'E:\\ComputerVision\\computer-vision\\3_1YOLO\\home\\VideoTobe\\ADemoV7 (5).mp4', 'E:\\ComputerVision\\computer-vision\\3_1YOLO\\home\\VideoTobe\\ADemoV7.mp4', 'E:\\ComputerVision\\computer-vision\\3_1YOLO\\home\\VideoTobe\\Demo1.mp4', 'E:\\ComputerVision\\computer-vision\\3_1YOLO\\home\\VideoTobe\\demo2.mp4', 'E:\\ComputerVision\\computer-vision\\3_1YOLO\\home\\VideoTobe\\Demo4.mp4']


In [1]:
#Import libraries
import cv2
import os
import numpy as np

#Reading the video
vidcap = cv2.VideoCapture(PathFoot)
success,image = vidcap.read()
count = 0
success = True
idx = 0

#Read the video frame by frame
while success:
    #converting into hsv image
    hsv = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    #green range
    lower_green = np.array([40,40, 40])
    upper_green = np.array([70, 255, 255])
    #blue range
    lower_blue = np.array([110,50,50])
    upper_blue = np.array([130,255,255])

    #Red range
    lower_red = np.array([0,31,255])
    upper_red = np.array([176,255,255])

    #white range
    lower_white = np.array([0,0,0])
    upper_white = np.array([0,0,255])

    #Define a mask ranging from lower to uppper
    mask = cv2.inRange(hsv, lower_green, upper_green)
    #Do masking
    res = cv2.bitwise_and(image, image, mask=mask)
    #convert to hsv to gray
    res_bgr = cv2.cvtColor(res,cv2.COLOR_HSV2BGR)
    res_gray = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)

    #Defining a kernel to do morphological operation in threshold image to 
    #get better output.
    kernel = np.ones((13,13),np.uint8)
    thresh = cv2.threshold(res_gray,127,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    

    #find contours in threshold image     
    im2,contours,hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)


    prev = 0
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)

        #Detect players
        if(h>=(1.5)*w):
            if(w>15 and h>= 15):
                idx = idx+1
                player_img = image[y:y+h,x:x+w]
                player_hsv = cv2.cvtColor(player_img,cv2.COLOR_BGR2HSV)
                #If player has blue jersy
                mask1 = cv2.inRange(player_hsv, lower_blue, upper_blue)
                res1 = cv2.bitwise_and(player_img, player_img, mask=mask1)
                res1 = cv2.cvtColor(res1,cv2.COLOR_HSV2BGR)
                res1 = cv2.cvtColor(res1,cv2.COLOR_BGR2GRAY)
                nzCount = cv2.countNonZero(res1)
                #If player has red jersy
                mask2 = cv2.inRange(player_hsv, lower_red, upper_red)
                res2 = cv2.bitwise_and(player_img, player_img, mask=mask2)
                res2 = cv2.cvtColor(res2,cv2.COLOR_HSV2BGR)
                res2 = cv2.cvtColor(res2,cv2.COLOR_BGR2GRAY)
                nzCountred = cv2.countNonZero(res2)

                if(nzCount >= 20):
                    #Mark blue jersy players as france
                    cv2.putText(image, 'France', (x-2, y-2), font, 0.8, (255,0,0), 2, cv2.LINE_AA)
                    cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),3)
                else:
                    pass
                if(nzCountred>=20):
                    #Mark red jersy players as belgium
                    cv2.putText(image, 'Belgium', (x-2, y-2), font, 0.8, (0,0,255), 2, cv2.LINE_AA)
                    cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),3)
                else:
                    pass
        if((h>=1 and w>=1) and (h<=30 and w<=30)):
            player_img = image[y:y+h,x:x+w]

            player_hsv = cv2.cvtColor(player_img,cv2.COLOR_BGR2HSV)
            #white ball  detection
            mask1 = cv2.inRange(player_hsv, lower_white, upper_white)
            res1 = cv2.bitwise_and(player_img, player_img, mask=mask1)
            res1 = cv2.cvtColor(res1,cv2.COLOR_HSV2BGR)
            res1 = cv2.cvtColor(res1,cv2.COLOR_BGR2GRAY)
            nzCount = cv2.countNonZero(res1)


            if(nzCount >= 3):
                # detect football
                cv2.putText(image, 'football', (x-2, y-2), font, 0.8, (0,255,0), 2, cv2.LINE_AA)
                cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),3)


    #cv2.imwrite("./Cropped/frame%d.jpg" % count, res)
    print ('Read a new frame: ', success )    # save frame as JPEG file	
    count += 1
    cv2.imshow('Match Detection',image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    success,image = vidcap.read()

vidcap.release()
cv2.destroyAllWindows()

NameError: name 'PathFoot' is not defined

In [3]:
class ShapeDetector:
    def __init__(self):
        pass

    def detect(self, c):
        # initialize the shape name and approximate the contour
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        if len(approx) == 3:
            shape = "triangle"

        # if the shape has 4 vertices, it is either a square or
        # a rectangle
        elif len(approx) == 4:
            # compute the bounding box of the contour and use the
            # bounding box to compute the aspect ratio
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)

            # a square will have an aspect ratio that is approximately
            # equal to one, otherwise, the shape is a rectangle
            shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"

        # if the shape is a pentagon, it will have 5 vertices
        elif len(approx) == 5:
            shape = "pentagon"

        # otherwise, we assume the shape is a circle
        else:
            shape = "circle"

        # return the name of the shape
        return shape
sd = ShapeDetector()

In [7]:
#Import libraries
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
#import imutils
#Reading the video
# vidcap = cv2.VideoCapture(pathFoot)
# success,image = vidcap.read()
# count = 0
# success = True
# idx = 0

for file in v:
    vidcap = cv2.VideoCapture(file)
    success,image = vidcap.read()
    count = 0
    success = True
    idx = 0
    #Read the video frame by frame
    while success:
        #converting into hsv image
        hsv = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
        #green range
        lower_green = np.array([40,40, 40])
        upper_green = np.array([70, 255, 255])
        #blue range
        lower_blue = np.array([110,50,50])
        upper_blue = np.array([130,255,255])

        #Red range
        lower_red = np.array([0,31,255])
        upper_red = np.array([176,255,255])

        #white range
        lower_white = np.array([0,0,0])
        upper_white = np.array([0,0,25])

        #not white
        low = np.array([15, 15, 150])
        high = np.array([250, 250, 250])



        #Define a mask ranging from lower to uppper
        mask = cv2.inRange(hsv, low,high)
        #Do masking
        res = cv2.bitwise_and(image, image, mask=mask)
        #convert to hsv to gray
        res_bgr = cv2.cvtColor(res,cv2.COLOR_HSV2BGR)
        res_gray = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
        # gaussian blur is applied to improve the edge detections
        res_gray = cv2.GaussianBlur(res_gray, (5, 5), 0)
        #Defining a kernel to do morphological operation in threshold image to 
        #get better output.
        kernel = np.ones((13,13),np.uint8)
        #thresh = cv2.threshold(res_gray,127,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        #thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
        thresh = cv2.threshold(res_gray, 60, 255, cv2.THRESH_BINARY)[1]
        print ('Read a new frame: ', success )    # save frame as JPEG file	
        count += 1
        
       

        # Using the Canny filter to get contours
        edges = cv2.Canny(res_gray, 1, 10)
        # Using the Canny filter with different parameters
        edges_high_thresh = cv2.Canny(res_gray, 60, 120)
        minLineLength = 1000
        maxLineGap = 5
        
        
        lines = cv2.HoughLinesP(res_gray,1,np.pi/180,3,minLineLength,maxLineGap)
        print(np.shape(lines))
        try:
            for l in lines:
            #print(np.shape(l) , l)
                x1,y1,x2,y2 = l[0,:]
                cv2.line(image,(x1,y1),(x2,y2),(0,255,255),2)
        except:
            print("unexcepted error" , lines)

#         thresh = cv2.threshold(image, 60, 255, cv2.THRESH_BINARY)[1]
#         cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#         #cnts = imutils.grab_contours(cnts)
#         # Stacking the images to print them together
#         # For comparison
#         #images = np.hstack((gray, edges, edges_high_thresh))
#         print(np.shape(cnts))
#         for c in cnts:
#             try:
#                 # compute the center of the contour, then detect the name of the
#                 # shape using only the contour
#                 M = cv2.moments(c)
#                 cX = int((M["m10"] / M["m00"]) )
#                 cY = int((M["m01"] / M["m00"]) )
#                 shape = sd.detect(c)
#                 print(shape)
#                 # multiply the contour (x, y)-coordinates by the resize ratio,
#                 # then draw the contours and the name of the shape on the image
#                 c = c.astype("float")
#                 c *= ratio
#                 c = c.astype("int")
#                 cv2.drawContours(image, [c], -1, (0, 255, 0), 2)
#                 cv2.putText(image, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
#             except:
#                 print("except the unexcepted")

   
        cv2.imshow('Match Detection',edges_high_thresh)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        success,image = vidcap.read()
        
    
vidcap.release()
cv2.destroyAllWindows()

Read a new frame:  True
(12314, 1, 4)
Read a new frame:  True
(12309, 1, 4)
Read a new frame:  True
(12679, 1, 4)
Read a new frame:  True
(12706, 1, 4)
Read a new frame:  True
(12751, 1, 4)
Read a new frame:  True
(12414, 1, 4)
Read a new frame:  True
(11798, 1, 4)
Read a new frame:  True
(11900, 1, 4)
Read a new frame:  True
(12158, 1, 4)
Read a new frame:  True
(11737, 1, 4)
Read a new frame:  True
(11953, 1, 4)
Read a new frame:  True
(12053, 1, 4)
Read a new frame:  True
(12491, 1, 4)
Read a new frame:  True
(12229, 1, 4)
Read a new frame:  True
(12060, 1, 4)
Read a new frame:  True
(10750, 1, 4)
Read a new frame:  True
(11787, 1, 4)
Read a new frame:  True
(12193, 1, 4)
Read a new frame:  True
(11729, 1, 4)
Read a new frame:  True
(12003, 1, 4)
Read a new frame:  True
(12603, 1, 4)
Read a new frame:  True
(13084, 1, 4)
Read a new frame:  True
(11584, 1, 4)
Read a new frame:  True
(12349, 1, 4)
Read a new frame:  True
(12015, 1, 4)
Read a new frame:  True
(12003, 1, 4)
Read a new f

(6095, 1, 4)
Read a new frame:  True
(6176, 1, 4)
Read a new frame:  True
(6140, 1, 4)
Read a new frame:  True
(6622, 1, 4)
Read a new frame:  True
(5871, 1, 4)
Read a new frame:  True
(5912, 1, 4)
Read a new frame:  True
(6201, 1, 4)
Read a new frame:  True
(5787, 1, 4)
Read a new frame:  True
(6311, 1, 4)
Read a new frame:  True
(5794, 1, 4)
Read a new frame:  True
(6081, 1, 4)
Read a new frame:  True
(5789, 1, 4)
Read a new frame:  True
(5617, 1, 4)
Read a new frame:  True
(5295, 1, 4)
Read a new frame:  True
(5345, 1, 4)
Read a new frame:  True
(4716, 1, 4)
Read a new frame:  True
(5058, 1, 4)
Read a new frame:  True
(4798, 1, 4)
Read a new frame:  True
(4739, 1, 4)
Read a new frame:  True
(3924, 1, 4)
Read a new frame:  True
(4627, 1, 4)
Read a new frame:  True
(4570, 1, 4)
Read a new frame:  True
(4109, 1, 4)
Read a new frame:  True
(4395, 1, 4)
Read a new frame:  True
(4313, 1, 4)
Read a new frame:  True
(4698, 1, 4)
Read a new frame:  True
(5008, 1, 4)
Read a new frame:  True
(

In [52]:
vidcap.release()
cv2.destroyAllWindows()